In [ ]:
"""
by Yudha Putra Arisandy
Departement of Computer Science
IPB University
Indonesia
"""

# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read dataframe for excel file
df = pd.read_excel('Data Pre-Riset.xlsx', sheet_name='main')
print(df.head(5))
print(df.isnull().sum())
df.describe().transpose()

In [ ]:
# Drop or cut the unnecessary column
df = df.drop('Panjang Gelombang (nm)',axis=1)
df = df.drop('N',axis=1)
df = df.drop('P',axis=1)

# create X and Y variable for training
X = df.drop('N',axis=1)
y = df['N']

# split data to 70:30
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
df.shape

In [ ]:
# Preprocessing Random Oversampling
# SMOGN
!pip install smogn

# load libraries
import smogn
import pandas
## conduct smogn
df_smogn = smogn.smoter(
    
    ## main arguments
    data = df,           ## pandas dataframe
    y = 'K',          ## string ('header name')
    k = 9,                    ## positive integer (k < n) , 9, 12
    samp_method = 'extreme',  ## string ('balance' or 'extreme')

    ## phi relevance arguments
    rel_thres = 0.99,         ## positive real number (0 < R < 1), 0.99
    rel_method = 'auto',      ## string ('auto' or 'manual')
    rel_xtrm_type = 'high',   ## string ('low' or 'both' or 'high')
    rel_coef = 0.9           ## positive real number (0 < R), 0.9, 3.6
)
df_smogn.to_csv('df_smogn_k.csv')
df_smogn.shape

In [ ]:
X = df_smogn.drop('N',axis=1)
y = df_smogn['N']

In [ ]:
#read df_smogn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('df_smogn_n.csv')
#print(df.head(5))
#print(df.isnull().sum())
df = df.drop('Unnamed: 0',axis=1)
df.describe().transpose()
X = df.drop('N',axis=1)
y = df['N']

In [ ]:
#Preprocessing: Principal Component Analysis (PCA)
# calculate matriks covariance
features = X.T
cov_matrix = np.cov(features)
cov_matrix[:5]

#Eigen Decomposition
values, vectors = np.linalg.eig(cov_matrix)
values=values.real
vectors=vectors.real
values[:5]

explained_variances = []
for i in range(len(values)):
 explained_variances.append(values[i] / np.sum(values))
print(np.sum(explained_variances), '\n', explained_variances)

projected_1 = X.dot(vectors.T[0])
projected_2 = X.dot(vectors.T[1])
projected_3 = X.dot(vectors.T[2])
projected_4 = X.dot(vectors.T[3])
projected_5 = X.dot(vectors.T[4])
projected_6 = X.dot(vectors.T[5])
projected_7 = X.dot(vectors.T[6])
projected_8 = X.dot(vectors.T[7])
projected_9 = X.dot(vectors.T[8])
projected_10 = X.dot(vectors.T[9])
df_pca = pd.DataFrame(projected_1, columns=['PC1'])
df_pca['PC2'] = projected_2
df_pca['PC3'] = projected_3
df_pca['PC4'] = projected_4
df_pca['PC5'] = projected_5
df_pca['PC6'] = projected_6
df_pca['PC7'] = projected_7
df_pca['PC8'] = projected_8
df_pca['PC9'] = projected_9
df_pca['PC10'] = projected_10
df_pca['N'] = y
df_pca.to_csv('df_pca10_n.csv')
df_pca

In [ ]:
# Plot data to scatterplot
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20, 10))
sns.scatterplot(df_pca['PC1'], [0] * len(df_pca), hue=df_pca['K'], s=200)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Variable to save the result in .csv
pc='raw'
unsur='k'

# choose one of these line below:
#df_pca = pd.read_csv('df_pca10_k.csv')
#df_pca = pd.read_excel('df_smogn_p.xlsx')
#df_pca = pd.read_csv('df_smogn_k.csv')
df_pca = pd.read_excel('df_raw_k.xlsx')

df_pca.drop('Unnamed: 0', axis=1, inplace=True)
df_pca.head(5)

# create X and Y variable for model
X = df_pca.drop('K',axis=1)
y = df_pca['K']

# split train and test data 80:20
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.head(5)

In [ ]:
#1. Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from sklearn.model_selection import GridSearchCV
parameters={
    'n_estimators': [100, 200, 400, 600, 800, 1000], #200, 400, 600, 800, 1000
    'max_depth': [None, 10, 30, 50], #10, 30, 50
    'min_samples_split':[2, 5], #2, 5
    'min_samples_leaf': [1, 3], #1, 3
    'bootstrap': [False, True]
    }

rf = RandomForestRegressor()
regr_rf = GridSearchCV(rf, parameters)
regr_rf.fit(X_train, y_train)

y_pred = regr_rf.predict(X_test)

r2=regr_rf.best_score_
rmse=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
mape=np.mean(np.abs((y_test - y_pred)/y_test))*100
nse=1-(np.sum((y_test-y_pred)**2)/np.sum((y_test-np.mean(y_pred))**2))
akurasi=100-np.mean(np.abs((y_test - y_pred)/y_test))*100
best_estimator=regr_rf.best_estimator_
best_param= regr_rf.best_params_

print("Random Forest")
print('R2     :', r2)
print('RMSE   :', rmse)
print('MAPE   :', mape)
print('NSE    :', nse)
print('AKURASI:', akurasi)
print("\n Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", best_estimator)
print("\n The best parameters across ALL searched params:\n", best_param)

#Simpan semua hasil statistik dalam file csv
import csv  
header1 = ['R2', 'RMSE', 'MAPE', 'NSE', 'AKURASI', 'Best Estimator', 'Best Parameter']
data1 = [r2, rmse, mape, nse, akurasi, best_estimator, best_param]
header2 = ['y_test']
data2 = [y_test]
header3 = ['y_pred']
data3 = [y_pred]

model='rf'
nama_file='data_statistik' + '_' + model + '_' + pc + '_' + unsur + '.csv'

with open(nama_file, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header1)
    writer.writerow(data1)
    writer.writerow("\n")
    writer.writerow(header2)
    writer.writerow(data2)
    writer.writerow("\n")
    writer.writerow(header3)
    writer.writerow(data3)

# Plot grafik regresi
plt.scatter(y_test,y_pred)
plt.plot(y_test,y_test,'r')

In [ ]:
#2. Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics

from sklearn.model_selection import GridSearchCV
parameters={
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0], #0.0 - inf
    'n_estimators': [1, 3, 5, 7, 10, 15, 20, 100], #1 - inf
    'subsample':[0.1, 0.3, 0.5, 0.7, 1], #0-1
    'max_depth': [1, 3, 5, 7, 10, 15], #1-inf
    }
gb = GradientBoostingRegressor()
regr_gb = GridSearchCV(gb, parameters)
regr_gb.fit(X_train, y_train)

y_pred = regr_gb.predict(X_test)

r2=regr_gb.best_score_
rmse=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
mape=np.mean(np.abs((y_test - y_pred)/y_test))*100
nse=1-(np.sum((y_test-y_pred)**2)/np.sum((y_test-np.mean(y_pred))**2))
akurasi=100-np.mean(np.abs((y_test - y_pred)/y_test))*100
best_estimator=regr_gb.best_estimator_
best_param= regr_gb.best_params_

print("Gradient Boosting")
print('R2     :', r2)
print('RMSE   :', rmse)
print('MAPE   :', mape)
print('NSE    :', nse)
print('AKURASI:', akurasi)
print("\n Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", best_estimator)
print("\n The best parameters across ALL searched params:\n", best_param)

#Simpan semua hasil statistik dalam file csv
import csv  
header1 = ['R2', 'RMSE', 'MAPE', 'NSE', 'AKURASI', 'Best Estimator', 'Best Parameter']
data1 = [r2, rmse, mape, nse, akurasi, best_estimator, best_param]
header2 = ['y_test']
data2 = [y_test]
header3 = ['y_pred']
data3 = [y_pred]

model='gb'
nama_file='data_statistik' + '_' + model + '_' + pc + '_' + unsur + '.csv'

with open(nama_file, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header1)
    writer.writerow(data1)
    writer.writerow("\n")
    writer.writerow(header2)
    writer.writerow(data2)
    writer.writerow("\n")
    writer.writerow(header3)
    writer.writerow(data3)

# Plot grafik regresi
plt.scatter(y_test,y_pred)
plt.plot(y_test,y_test,'r')

In [ ]:
#3 Support Vector Machine
from sklearn.svm import SVR
from sklearn import metrics

from sklearn.model_selection import GridSearchCV
parameters={
    'C': [1.0, 2.0, 5.0, 7.0, 10.0], #positif
    'epsilon': [0.1, 0.3, 0.5, 0.7, 1.0],
    'gamma':['scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }
svr = SVR()
regr_svr = GridSearchCV(svr, parameters)
regr_svr.fit(X_train, y_train)

y_pred = regr_svr.predict(X_test)

r2=regr_svr.best_score_
rmse=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
mape=np.mean(np.abs((y_test - y_pred)/y_test))*100
nse=1-(np.sum((y_test-y_pred)**2)/np.sum((y_test-np.mean(y_pred))**2))
akurasi=100-np.mean(np.abs((y_test - y_pred)/y_test))*100
best_estimator=regr_svr.best_estimator_
best_param= regr_svr.best_params_

print("Support Vector Machine")
print('R2     :', r2)
print('RMSE   :', rmse)
print('MAPE   :', mape)
print('NSE    :', nse)
print('AKURASI:', akurasi)
print("\n Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", best_estimator)
print("\n The best parameters across ALL searched params:\n", best_param)

#Simpan semua hasil statistik dalam file csv
import csv  
header1 = ['R2', 'RMSE', 'MAPE', 'NSE', 'AKURASI', 'Best Estimator', 'Best Parameter']
data1 = [r2, rmse, mape, nse, akurasi, best_estimator, best_param]
header2 = ['y_test']
data2 = [y_test]
header3 = ['y_pred']
data3 = [y_pred]

model='svm'
nama_file='data_statistik' + '_' + model + '_' + pc + '_' + unsur + '.csv'

with open(nama_file, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header1)
    writer.writerow(data1)
    writer.writerow("\n")
    writer.writerow(header2)
    writer.writerow(data2)
    writer.writerow("\n")
    writer.writerow(header3)
    writer.writerow(data3)

# Plot grafik regresi
plt.scatter(y_test,y_pred)
plt.plot(y_test,y_test,'r')

In [ ]:
#4. PLS
from sklearn.cross_decomposition import PLSRegression
from sklearn import metrics

from sklearn.model_selection import GridSearchCV
parameters={
    'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 40, 60, 80, 100, 120],
    'max_iter' : [300, 500, 1000]
    }
pls = PLSRegression()
regr_pls = GridSearchCV(pls, parameters)
regr_pls.fit(X_train, y_train)

y_pred = regr_pls.predict(X_test)

r2=regr_pls.best_score_
rmse=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
mape=np.mean(np.abs((y_test.ravel() - y_pred.ravel())/y_test.ravel()))*100
nse=1-(np.sum((y_test.ravel()-y_pred.ravel())**2)/np.sum((y_test.ravel()-np.mean(y_pred.ravel()))**2))
akurasi=100-np.mean(np.abs((y_test.ravel() - y_pred.ravel())/y_test.ravel()))*100
best_estimator=regr_pls.best_estimator_
best_param= regr_pls.best_params_

print("Partial Least Square")
print('R2     :', r2)
print('RMSE   :', rmse)
print('MAPE   :', mape)
print('NSE    :', nse)
print('AKURASI:', akurasi)
print("\n Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", best_estimator)
print("\n The best parameters across ALL searched params:\n", best_param)

#Simpan semua hasil statistik dalam file csv
import csv  
header1 = ['R2', 'RMSE', 'MAPE', 'NSE', 'AKURASI', 'Best Estimator', 'Best Parameter']
data1 = [r2, rmse, mape, nse, akurasi, best_estimator, best_param]
header2 = ['y_test']
data2 = [y_test]
header3 = ['y_pred']
data3 = [y_pred]

model='pls'
nama_file='data_statistik' + '_' + model + '_' + pc + '_' + unsur + '.csv'

with open(nama_file, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header1)
    writer.writerow(data1)
    writer.writerow("\n")
    writer.writerow(header2)
    writer.writerow(data2)
    writer.writerow("\n")
    writer.writerow(header3)
    writer.writerow(data3)

# Plot grafik regresi
plt.scatter(y_test,y_pred)
plt.plot(y_test,y_test,'r')

In [ ]:
#5. Deep Neural Network

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow.keras as keras

# deeper or wider
regr_dnn = Sequential()
regr_dnn.add(Dense(1024, input_dim=136, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(1024, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(512, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(512, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(512, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(256, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(256, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(256, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(128, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(128, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(128, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(64, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(64, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(64, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(32, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(32, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(32, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(16, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(16, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(16, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(8, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(8, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(8, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(4, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(4, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(4, kernel_initializer='normal', activation='relu'))
regr_dnn.add(Dense(1, kernel_initializer='normal'))
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
regr_dnn.compile(optimizer=optimizer,loss='mse')

regr_dnn.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=8,epochs=1500)

from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score, mean_absolute_percentage_error
y_pred = regr_dnn.predict(X_test)

from sklearn.metrics import r2_score
r2=r2_score(y_test, y_pred)
rmse=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
mape=np.mean(np.abs((y_test.ravel() - y_pred.ravel())/y_test.ravel()))*100
nse=1-(np.sum((y_test.ravel()-y_pred.ravel())**2)/np.sum((y_test.ravel()-np.mean(y_pred.ravel()))**2))
akurasi=100-np.mean(np.abs((y_test.ravel() - y_pred.ravel())/y_test.ravel()))*100

print("Deep Neural Network")
print('R2     :', r2)
print('RMSE   :', rmse)
print('MAPE   :', mape)
print('NSE    :', nse)
print('AKURASI:', akurasi)

#Simpan semua hasil statistik dalam file csv
import csv  
header1 = ['R2', 'RMSE', 'MAPE', 'NSE', 'AKURASI']
data1 = [r2, rmse, mape, nse, akurasi]
header2 = ['y_test']
data2 = [y_test]
header3 = ['y_pred']
data3 = [y_pred]

model='dnn'
nama_file='data_statistik' + '_' + model + '_' + pc + '_' + unsur + '.csv'

with open(nama_file, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header1)
    writer.writerow(data1)
    writer.writerow("\n")
    writer.writerow(header2)
    writer.writerow(data2)
    writer.writerow("\n")
    writer.writerow(header3)
    writer.writerow(data3)

# Plot grafik regresi
plt.scatter(y_test,y_pred)
plt.plot(y_test,y_test,'r')